In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb as dd

# Load the data

pd.DataFrame.

In [ ]:
#read data from C:\Projects\GitHub\stldata.io\csb_data\merged_csb_data.csv
df = pd.read_csv('C:/Projects/GitHub/stldata.io/csb_data/merged_csb_data.csv',dtype=str)
df['DATETIMECLOSED'] = pd.to_datetime(df['DATETIMECLOSED'])
df['DATEINVTDONE'] = pd.to_datetime(df['DATETIMECLOSED'])
df['DATETIMEINIT'] = pd.to_datetime(df['DATETIMEINIT'])
df['DATECANCELLED'] = pd.to_datetime(df['DATECANCELLED'])
df.head()

,CALLERTYPE,CITY,DATECANCELLED,DATEINVTDONE,DATETIMECLOSED,DATETIMEINIT,DESCRIPTION,EXPLANATION,GRANDPARENT_ID,GRANDPARENT_NODE,...,PROBLEMCODE,PROBLEMSID,PROBZIP,PUBLICRESOLUTION,REQUESTID,SRX,SRY,STATUS,SUBMITTO,WARD
0,INDETERMINATE,St. Louis,NaN,2021-01-01 08:09:04.0,2021-01-01 08:09:37.217,2021-01-01 01:24:46.0,Signal Dark,NaN,1874,Citizen Requests,...,Signal Dark,309,NaN,work completed,1423457,-10051407.4428938,4661984.1440219,CLOSED,"TRAFFIC DIVISION, 511",5.0
1,INDETERMINATE,St. Louis,NaN,2021-01-04 06:48:40.413,2021-01-04 06:48:40.413,2021-01-01 01:26:35.0,Whole block lights out,Every street light on the block is dark.,1874,Citizen Requests,...,Whole Blk Lights Out,1730,NaN,repairs made,1423458,-10052110.915226,4661979.4236312,CLOSED,"TRAFFIC DIVISION, 511",2.0
2,INDETERMINATE,St. Louis,NaN,2021-01-04 15:05:26.0,2021-01-04 15:07:22.003,2021-01-01 01:33:53.0,Bad Condition,NaN,1187,Right of Way Issues,...,Bad Condition,363,NaN,removed and made safe,1423459,-10042323.5786453,4674253.932627,CLOSED,"STREET DEPT, 510",11.0
3,INDETERMINATE,St. Louis,NaN,2021-01-01 08:57:54.0,2021-01-01 08:58:34.227,2021-01-01 01:42:36.0,Whole block lights out,Every street light on the block is dark.,1874,Citizen Requests,...,Whole Blk Lights Out,1730,NaN,no problem found,1423460,-10044161.8193057,4672511.059541,CLOSED,"TRAFFIC DIVISION, 511",12.0
4,INDETERMINATE,St. Louis,NaN,2021-01-01 06:29:00.0,2021-04-05 08:50:46.063,2021-01-01 02:21:04.0,Light Damaged,"Street light damaged, including any damage to ...",1874,Citizen Requests,...,Light Damaged,319,NaN,work completed,1423461,-10039813.7425135,4669408.5168311,CLOSED,"TRAFFIC DIVISION, 511",14.0


In [63]:
df.dtypes

CALLERTYPE                                    object
CITY                                          object
DATECANCELLED                                 object
DATEINVTDONE                                  object
DATETIMECLOSED                        datetime64[ns]
DATETIMEINIT                                  object
DESCRIPTION                                   object
EXPLANATION                                   object
GRANDPARENT_ID                                object
GRANDPARENT_NODE                              object
GROUP                                         object
NEIGHBORHOOD                                  object
PARENT_ID                                     object
PARENT_NODE                                   object
PLAIN_ENGLISH_NAME_FOR_PROBLEMCODE            object
PRJCOMPLETEDATE                               object
PROBADDRESS                                   object
PROBADDTYPE                                   object
PROBLEMCODE                                   